In [1]:
using Revise

In [8]:
using RigidBodyDynamics
using RigidBodyDynamics.Contact
using MeshCat
using MeshCatMechanisms
using AtlasRobot
using Blink
using RigidBodySim

INFO: Recompiling stale cache file /home/rdeits/locomotion/explorations/learning-mpc-2/packages/lib/v0.6/LCPSim.ji for module LCPSim.


In [3]:
vis = Visualizer()
open(vis, Window())
manip_window = Window()

Blink.AtomShell.Window(2, Blink.AtomShell.Electron(Process(`/home/rdeits/locomotion/explorations/learning-mpc-2/packages/v0.6/Blink/deps/atom/electron /home/rdeits/locomotion/explorations/learning-mpc-2/packages/v0.6/Blink/src/AtomShell/main.js port 8168`, ProcessRunning), TCPSocket(RawFD(63) active, 0 bytes waiting), Dict{String,Any}(Pair{String,Any}("callback", Blink.#3))), Blink.Page(2, #undef, Dict{String,Any}(Pair{String,Any}("callback", Blink.#3)), Future(1, 1, 2, Nullable{Any}())))

In [4]:
delete!(vis)
mechanism = parse_urdf(Float64, "atlas.urdf")
pelvis = findbody(mechanism, "pelvis")
basejoint = joint_to_parent(pelvis, mechanism)
floatingjoint = Joint(basejoint.name, frame_before(basejoint), frame_after(basejoint),
    QuaternionFloating{Float64}())
replace_joint!(mechanism, basejoint, floatingjoint)
basejoint = floatingjoint

contactmodel = SoftContactModel(hunt_crossley_hertz(k = 500e3), ViscoelasticCoulombModel(0.8, 20e3, 100.))
for (name, offset) in [
        ("l_lleg", [0, 0, -0.452]),
        ("r_lleg", [0, 0, -0.452]),
        ("l_ufarm", [0, -0.05, 0]),
        ("r_ufarm", [0, -0.05, 0])
    ]
    body = findbody(mechanism, name)
    frame = default_frame(body)
    add_contact_point!(body, ContactPoint(Point3D(frame, offset...), contactmodel))
end

add_environment_primitive!(mechanism, HalfSpace3D(Point3D(root_frame(mechanism), 0., 0, 0),
        FreeVector3D(root_frame(mechanism), 0., 0, 1)))

mvis = MechanismVisualizer(mechanism, URDFVisuals("atlas.urdf", package_path=[AtlasRobot.packagepath()]), vis)
body!(manip_window, manipulate!(mvis))

Blink.Page(2, WebSockets.WebSocket{TCPSocket}(TCPSocket(RawFD(64) active, 0 bytes waiting), true, CONNECTED::WebSockets.ReadyState = 1), Dict{String,Any}(Pair{String,Any}("webio", WebIO.#111),Pair{String,Any}("callback", Blink.#3)), Future(1, 1, 2, Nullable{Any}(true)))

In [5]:
function init!(state::MechanismState)
    m = state.mechanism
    kneebend = 1.1
    hipbendextra = 0.1
    for sideprefix in ('l', 'r')
        knee = findjoint(m, "$(sideprefix)_leg_kny")
        hippitch = findjoint(m, "$(sideprefix)_leg_hpy")
        shoulderroll = findjoint(m, "$(sideprefix)_arm_shx")
        set_configuration!(state, knee, [kneebend])
        set_configuration!(state, hippitch, [-kneebend / 2 + hipbendextra])
        if sideprefix == 'r'
            set_configuration!(state, shoulderroll, 1)
        else
            set_configuration!(state, shoulderroll, -1)
        end
    end
    set_configuration!(state, findjoint(m, "pelvis_to_world"), [1, 0, 0, 0, 0, 0, 0.79])
    state
end

init! (generic function with 1 method)

In [6]:
copy!(mvis, init!(MechanismState(mechanism)))

In [9]:
using LCPSim
using LearningMPC

INFO: Recompiling stale cache file /home/rdeits/locomotion/explorations/learning-mpc-2/packages/lib/v0.6/LearningMPC.ji for module LearningMPC.


In [11]:
size(Q)

(37, 37)

In [12]:
size(R)

(18, 18)

In [13]:
num_positions(state) + num_velocities(state)

37

### TODO:

I'm assuming in contact_lqr that the contact Jacobian is the same for positions and velocities. I probably need to multiply by J_qdot_to_v. 

In [26]:
qq = fill(0.1, num_positions(mechanism) + num_velocities(mechanism))
qq[1:num_positions(mechanism)] .= 1
qq[1:4] .= 100
qq[5:7] .= 100
Q = diagm(qq)
R = diagm(fill(1e-6, num_velocities(mechanism)))
Δt = 0.01
contacts = [
    Point3D(default_frame(findbody(mechanism, name)), offset...)
    for (name, offset) in [
        ("l_lleg", [0, 0, -0.452]),
        ("r_lleg", [0, 0, -0.452])
    ]
]
lqrsol = LQRSolution(init!(MechanismState(mechanism)),
    Q, R, Δt, contacts
)

LoadError: [91mDimensionMismatch("A has dimensions (25,37) but B has dimensions (36,37)")[39m

size(M) = (12, 37)
size(N) = (37, 25)
size(A) = (36, 37)
size(B) = (36, 18)


In [7]:
state = init!(MechanismState(mechanism))
dynamics = Dynamics(mechanism)
problem = ODEProblem(dynamics, state, (0., 1.))
sol = solve(problem, Tsit5(), abs_tol=1e-8, dt=1e-6)
setanimation!(mvis, sol)

true